## Sign Up Sheet Code

You'll need to run the first cell to initiate stuff before you do anything.

In [1]:
# Import dependencies and load in current data
import pandas as pd
import win32com.client

in_person_df = pd.read_csv('SignUpInPerson.csv')
virtual_df = pd.read_csv('SignUpVirtual.csv')

To sign someone up for a session:

In [2]:
# Update the status of someone who's signing up
# Information:
email = 'Cade_Isbell@jrbt.com'
new_status = 1
# Enter 1 for the 10a meeting and 2 for the 1p meeting

In [3]:
# Update the tables
if in_person_df['Status'].value_counts()[new_status] < 60:
    spot = in_person_df[in_person_df['Email'] == email].index[0]
    in_person_df.iloc[spot,2] = new_status
    print(in_person_df.iloc[spot,:])
else:
    print("Sorry, buddy; that'n's full.")

Name               Cade Isbell
Email     Cade_Isbell@jrbt.com
Status                       1
Name: 14, dtype: object


In [4]:
# Check the numbers

pd.DataFrame(in_person_df['Status'].value_counts())

,count
Status,
1,53
2,39
4,11
0,6
3,2


In [5]:
# Save the tables as csvs

in_person_df.to_csv('SignUpInPerson.csv',index=False)
virtual_df.to_csv('SignUpVirtual.csv',index=False)

in_person_df.rename(columns={'Status':'Session'}).replace(0,'Not signed up').replace(1,'10 am').replace(2, '1 pm').replace(3, 'Unable to attend').replace(4,'Will attend virtual').to_csv('Attendees_InPerson.csv',index=False)
# This one will be easier for humans to read

In [6]:
# Send a confirmation email to the person who signed up.

ol=win32com.client.Dispatch("outlook.application")

olmailitem=0x0 #size of the new email. Not entirely sure what this means, honestly

newmail = ol.CreateItem(olmailitem)
newmail.Subject = 'Confirmation for Sign-Up'
newmail.To = email
# newmail.CC = 'xyz@example.com'

bodytext = "You're signed up for "

if new_status == 1:
    bodytext += 'the 10:00 a.m. Civil Rights/EEO training session on 28 October.'
elif new_status == 2:
    bodytext += 'the 1:00 p.m. Civil Rights/EEO training session on 28 October.'

bodytext += " Both sessions will take place in the fourth floor conference room.\n\nThanks for getting back to us!\n\nThanks,\nJamey Yadon"

newmail.Body = bodytext

# Display the mail before sending it
newmail.Display() 

# This cell adapted from https://www.makeuseof.com/send-outlook-emails-using-python/

When it comes time to send the reminder to those who haven't signed up:

In [21]:
# Build the string of people to send to

email_to = ''

for i in range(len(in_person_df)):
    if in_person_df['Status'][i] == 0:
        email_to += in_person_df['Email'][i] + ';'

print(email_to[:50])

Alvin_Packard@jrbt.com;Brian_Bivona@jrbt.com;Bryan


In [22]:
# Send the email

ol=win32com.client.Dispatch("outlook.application")

olmailitem=0x0 #size of the new email. Not entirely sure what this means, honestly

newmail = ol.CreateItem(olmailitem)
newmail.Subject = 'Reminder about Civil Rights/EEO Training'
newmail.To = email_to

bodytext = """Good morning,

If you're receiving this email, I don't have you down for either session of the civil rights/EEO training on 28 October. Please email me and let me know whether you'd like to attend the 10am or the 1pm session, or if you will be unable to attend either. On Monday, we will assign a session to those who haven't signed up. Please also remember that the training is required, though accomodations can be made for those unable to attend.

Thank you,
Jamey L. Yadon
"""

newmail.Body = bodytext

# Display the mail before sending it
newmail.Display() 

# This cell adapted from https://www.makeuseof.com/send-outlook-emails-using-python/

In [8]:
virtual_list = in_person_df[in_person_df['Status'] == 3]['Email'].drop(73)

In [9]:
for email in virtual_list:
    spot = in_person_df[in_person_df['Email'] == email].index[0]
    in_person_df.iloc[spot,2] = 4

In [20]:
in_person_df[in_person_df['Status'] == 3]

,Name,Email,Status
31,Garry Marshall,Garry_Marshall@jrbt.com,3
73,Michael Brown,Michael_Brown@jrbt.com,3
